# 📊 Merge Results - Gabungkan Hasil Semua Member

Jalankan notebook ini **SETELAH** semua 4 member selesai training.

Notebook ini akan:
1. Mengumpulkan hasil dari semua model
2. Membuat perbandingan
3. Generate visualisasi
4. Membuat summary untuk report

---

In [ ]:
# ============================================================
# SETUP
# ============================================================

from google.colab import drive
drive.mount('/content/drive')

import os
import sys
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

PROJECT_PATH = '/content/drive/MyDrive/AlexNet_iFood2019'
CHECKPOINT_DIR = os.path.join(PROJECT_PATH, 'checkpoints')
EVAL_DIR = os.path.join(PROJECT_PATH, 'evaluation_results')
ANALYSIS_DIR = os.path.join(PROJECT_PATH, 'analysis_results')

os.makedirs(ANALYSIS_DIR, exist_ok=True)

MODELS = ['alexnet_baseline', 'alexnet_mod1', 'alexnet_mod2', 'alexnet_combined']
MEMBER_NAMES = ['Member 1', 'Member 2', 'Member 3', 'Member 4']

print("✅ Setup complete")

In [ ]:
# ============================================================
# CEK KELENGKAPAN FILE
# ============================================================

print("📁 Checking files from all members...\n")

all_complete = True

for i, model in enumerate(MODELS):
    history_file = os.path.join(CHECKPOINT_DIR, f'{model}_history.json')
    metrics_file = os.path.join(EVAL_DIR, f'{model}_val_metrics.json')
    
    h_exists = os.path.exists(history_file)
    m_exists = os.path.exists(metrics_file)
    
    status = "✅" if (h_exists and m_exists) else "❌"
    print(f"{status} {MEMBER_NAMES[i]}: {model}")
    print(f"   History: {'✓' if h_exists else '✗'}  Metrics: {'✓' if m_exists else '✗'}")
    
    if not (h_exists and m_exists):
        all_complete = False

print()
if all_complete:
    print("🎉 Semua file lengkap! Lanjutkan ke cell berikutnya.")
else:
    print("⚠️  Ada file yang belum lengkap. Pastikan semua member sudah selesai training.")

In [ ]:
# ============================================================
# LOAD SEMUA HASIL
# ============================================================

histories = {}
metrics = {}

for model in MODELS:
    # Load history
    history_file = os.path.join(CHECKPOINT_DIR, f'{model}_history.json')
    if os.path.exists(history_file):
        with open(history_file) as f:
            histories[model] = json.load(f)
    
    # Load metrics
    metrics_file = os.path.join(EVAL_DIR, f'{model}_val_metrics.json')
    if os.path.exists(metrics_file):
        with open(metrics_file) as f:
            metrics[model] = json.load(f)

print(f"Loaded {len(histories)} histories, {len(metrics)} metrics")

In [ ]:
# ============================================================
# TABEL PERBANDINGAN
# ============================================================

results = []

for i, model in enumerate(MODELS):
    row = {
        'Member': MEMBER_NAMES[i],
        'Model': model.replace('alexnet_', '').upper()
    }
    
    if model in histories:
        h = histories[model]
        row['Best Val Acc'] = max(h['val_acc'])
        row['Best Epoch'] = h['val_acc'].index(max(h['val_acc'])) + 1
    
    if model in metrics:
        m = metrics[model]
        row['Test Acc'] = m.get('accuracy', 0)
        row['Top-5 Acc'] = m.get('top5_accuracy', 0)
        row['Macro F1'] = m.get('macro_f1', 0)
    
    results.append(row)

df = pd.DataFrame(results)

print("=" * 80)
print("📊 PERBANDINGAN HASIL SEMUA MEMBER")
print("=" * 80)
print(df.to_string(index=False))
print("=" * 80)

# Best model
if 'Best Val Acc' in df.columns:
    best_idx = df['Best Val Acc'].idxmax()
    print(f"\n🏆 Model Terbaik: {df.iloc[best_idx]['Model']} ({df.iloc[best_idx]['Member']})")
    print(f"   Accuracy: {df.iloc[best_idx]['Best Val Acc']:.4f}")

# Save
df.to_csv(os.path.join(ANALYSIS_DIR, 'team_results_comparison.csv'), index=False)
print(f"\n✅ Saved to: {ANALYSIS_DIR}/team_results_comparison.csv")

In [ ]:
# ============================================================
# TRAINING CURVES COMPARISON
# ============================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

# Loss curves
ax = axes[0, 0]
for (model, hist), color in zip(histories.items(), colors):
    epochs = range(1, len(hist['train_loss']) + 1)
    ax.plot(epochs, hist['train_loss'], color=color, label=model, linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Training Loss')
ax.legend()
ax.grid(alpha=0.3)

ax = axes[0, 1]
for (model, hist), color in zip(histories.items(), colors):
    epochs = range(1, len(hist['val_loss']) + 1)
    ax.plot(epochs, hist['val_loss'], color=color, label=model, linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Validation Loss')
ax.legend()
ax.grid(alpha=0.3)

# Accuracy curves
ax = axes[1, 0]
for (model, hist), color in zip(histories.items(), colors):
    epochs = range(1, len(hist['train_acc']) + 1)
    ax.plot(epochs, hist['train_acc'], color=color, label=model, linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax.set_title('Training Accuracy')
ax.legend()
ax.grid(alpha=0.3)

ax = axes[1, 1]
for (model, hist), color in zip(histories.items(), colors):
    epochs = range(1, len(hist['val_acc']) + 1)
    ax.plot(epochs, hist['val_acc'], color=color, label=model, linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax.set_title('Validation Accuracy')
ax.legend()
ax.grid(alpha=0.3)

plt.suptitle('Training Curves - All Models', fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(ANALYSIS_DIR, 'all_models_training_curves.png'), dpi=200)
plt.show()

print(f"✅ Saved: all_models_training_curves.png")

In [ ]:
# ============================================================
# METRICS BAR CHART
# ============================================================

fig, ax = plt.subplots(figsize=(12, 6))

metric_names = ['accuracy', 'top5_accuracy', 'macro_f1']
metric_labels = ['Accuracy', 'Top-5 Accuracy', 'Macro F1']
x = np.arange(len(MODELS))
width = 0.25

for i, (metric, label) in enumerate(zip(metric_names, metric_labels)):
    values = [metrics[m].get(metric, 0) for m in MODELS if m in metrics]
    bars = ax.bar(x + i*width, values, width, label=label)
    
    for bar, val in zip(bars, values):
        ax.annotate(f'{val:.3f}',
                   xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                   xytext=(0, 3), textcoords='offset points',
                   ha='center', fontsize=9)

ax.set_xlabel('Model')
ax.set_ylabel('Score')
ax.set_title('Model Performance Comparison')
ax.set_xticks(x + width)
ax.set_xticklabels([m.replace('alexnet_', '').upper() for m in MODELS])
ax.legend()
ax.set_ylim(0, 1.1)
ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(os.path.join(ANALYSIS_DIR, 'all_models_metrics_comparison.png'), dpi=200)
plt.show()

print(f"✅ Saved: all_models_metrics_comparison.png")

In [ ]:
# ============================================================
# GENERATE REPORT DATA
# ============================================================

report_md = f"""# AlexNet iFood2019 - Team Results

## Team Members & Models

| Member | Model | Best Val Acc | Top-5 Acc | Macro F1 |
|--------|-------|--------------|-----------|----------|
"""

for i, model in enumerate(MODELS):
    if model in histories and model in metrics:
        h = histories[model]
        m = metrics[model]
        report_md += f"| {MEMBER_NAMES[i]} | {model.replace('alexnet_', '').upper()} | {max(h['val_acc']):.4f} | {m['top5_accuracy']:.4f} | {m['macro_f1']:.4f} |\n"

# Best model
best_model = max(metrics, key=lambda x: metrics[x]['accuracy'])
best_acc = metrics[best_model]['accuracy']

report_md += f"""
## Best Model

🏆 **{best_model}** with accuracy: **{best_acc:.4f}**

## Visualizations

- Training Curves: `all_models_training_curves.png`
- Metrics Comparison: `all_models_metrics_comparison.png`
"""

# Save
with open(os.path.join(ANALYSIS_DIR, 'team_results_summary.md'), 'w') as f:
    f.write(report_md)

print(report_md)
print(f"\n✅ Saved: team_results_summary.md")

In [ ]:
# ============================================================
# SELESAI!
# ============================================================

print("\n" + "=" * 60)
print("🎉 MERGE RESULTS COMPLETE!")
print("=" * 60)
print(f"\nSemua hasil tersimpan di: {ANALYSIS_DIR}")
print("\nFile yang dihasilkan:")
print("  - team_results_comparison.csv")
print("  - team_results_summary.md")
print("  - all_models_training_curves.png")
print("  - all_models_metrics_comparison.png")
print("\nGunakan file-file ini untuk mengisi FINAL_REPORT.md")
print("=" * 60)